# 간단한 메모리: 대화 기록 유지하기

이 노트북에서는 **AI와의 대화 기록을 유지**하는 가장 기본적인 방법을 배웁니다.

## 왜 메모리가 필요할까요?

### AI의 기본 특성: 기억을 못함

```
┌────────────────────────────────────────────────────────────────────┐
│                    AI는 기본적으로 기억이 없다!                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   대화 1:                                                          │
│   사용자: "내 이름은 민혁이야"                                     │
│   AI: "안녕하세요 민혁님!"                                         │
│                                                                    │
│   대화 2 (새로운 요청):                                            │
│   사용자: "내 이름이 뭐야?"                                        │
│   AI: "죄송합니다, 저는 당신의 이름을 모릅니다" 😓                 │
│                                                                    │
│   → 각 요청은 완전히 독립적으로 처리됨!                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 메모리의 역할

```
┌────────────────────────────────────────────────────────────────────┐
│                    메모리가 있으면                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   대화 기록 저장:                                                  │
│   [                                                                │
│     {"user": "내 이름은 민혁이야"},                                │
│     {"ai": "안녕하세요 민혁님!"},                                   │
│     {"user": "내 이름이 뭐야?"}   ← 새 질문                        │
│   ]                                                                │
│                                                                    │
│   AI에게 전체 기록 전달 → AI가 맥락 파악                           │
│                                                                    │
│   AI: "당신의 이름은 민혁입니다!" ✅                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. placeholder를 사용한 메모리 구현

## placeholder란?

**placeholder**는 "자리 표시자"입니다. 대화 기록이 들어갈 자리를 미리 만들어둡니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    placeholder의 역할                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   프롬프트 템플릿:                                                 │
│   ┌────────────────────────────────────────┐                       │
│   │ System: 당신은 친절한 어시스턴트입니다  │                       │
│   │                                        │                       │
│   │ ▼▼▼ {messages} placeholder ▼▼▼        │  ← 여기에 대화 기록    │
│   │                                        │                       │
│   └────────────────────────────────────────┘                       │
│                                                                    │
│   실행 시:                                                         │
│   ┌────────────────────────────────────────┐                       │
│   │ System: 당신은 친절한 어시스턴트입니다  │                       │
│   │ Human: 나는 프로그래밍을 좋아해요      │                       │
│   │ AI: J'adore programmer.               │                       │
│   │ Human: 뭐라고 말했죠?                 │  ← 대화 기록이 채워짐  │
│   └────────────────────────────────────────┘                       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# 프롬프트 템플릿 생성
# 'placeholder'는 대화 기록이 들어갈 자리를 만들어줍니다
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 어시스턴트입니다. 모든 질문에 최선을 다해 답하세요.'),
    ('placeholder', '{messages}'),  # 여기에 대화 기록이 들어감
])

# LLM 모델
model = ChatOllama(model='llama3.2')

# 체인 생성
chain = prompt | model

print("✅ 메모리 체인 생성 완료")

# 3. 대화 기록을 포함한 실행

이전 대화 내용을 함께 전달하여 AI가 맥락을 이해하게 합니다.

In [ ]:
# 대화 기록과 함께 실행
response = chain.invoke({
    'messages': [
        ('human', '다음 한국어 문장을 프랑스어로 번역하세요: 나는 프로그래밍을 좋아해요.'),
        ('ai', "J'adore programmer."),
        ('human', '뭐라고 말했죠?'),  # 이전 대화를 기억해야 답할 수 있음
    ],
})

print("=== AI 응답 ===")
print(response.content)

# 4. 메모리의 효과 확인

대화 기록 없이 같은 질문을 하면 어떻게 될까요?

In [ ]:
# 대화 기록 없이 실행 (메모리 없음)
response_no_memory = chain.invoke({
    'messages': [
        ('human', '뭐라고 말했죠?'),  # 맥락 없이 질문
    ],
})

print("=== 메모리 없이 응답 ===")
print(response_no_memory.content)
print("\n→ 맥락을 모르니까 뭘 물어보는지 모름!")

# 5. 대화 시뮬레이션

In [ ]:
# 대화 기록을 저장할 리스트
conversation_history = []

def chat(user_message):
    """
    사용자 메시지를 받아서 AI 응답을 반환하고
    대화 기록을 업데이트합니다.
    """
    # 사용자 메시지 추가
    conversation_history.append(('human', user_message))
    
    # AI 응답 생성 (전체 대화 기록 전달)
    response = chain.invoke({'messages': conversation_history})
    
    # AI 응답을 기록에 추가
    conversation_history.append(('ai', response.content))
    
    return response.content

print("✅ 대화 함수 준비 완료")

In [ ]:
# 대화 시뮬레이션
print("=== 대화 시작 ===")

# 첫 번째 대화
print("\n사용자: 안녕하세요! 제 이름은 민혁입니다.")
response1 = chat("안녕하세요! 제 이름은 민혁입니다.")
print(f"AI: {response1}")

# 두 번째 대화
print("\n사용자: 저는 파이썬 프로그래밍을 배우고 있어요.")
response2 = chat("저는 파이썬 프로그래밍을 배우고 있어요.")
print(f"AI: {response2}")

# 세 번째 대화 (이전 내용 기억 확인)
print("\n사용자: 제 이름이 뭐였죠?")
response3 = chat("제 이름이 뭐였죠?")
print(f"AI: {response3}")

In [ ]:
# 현재 대화 기록 확인
print("=== 현재 대화 기록 ===")
for i, (role, content) in enumerate(conversation_history):
    role_name = "사용자" if role == "human" else "AI"
    print(f"[{i+1}] {role_name}: {content[:50]}..." if len(content) > 50 else f"[{i+1}] {role_name}: {content}")

---

## 정리: 간단한 메모리

### 핵심 원리

```
┌────────────────────────────────────────────────────────────────────┐
│                    메모리의 핵심 원리                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   AI는 기본적으로 기억이 없다                                      │
│   → 우리가 대화 기록을 직접 관리해야 한다                          │
│   → 매번 요청할 때 전체 대화 기록을 함께 전달                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. placeholder로 대화 기록 자리 만들기
prompt = ChatPromptTemplate.from_messages([
    ('system', '시스템 메시지'),
    ('placeholder', '{messages}'),  # 대화 기록이 들어갈 자리
])

# 2. 대화 기록과 함께 실행
response = chain.invoke({
    'messages': [
        ('human', '첫 번째 질문'),
        ('ai', '첫 번째 답변'),
        ('human', '두 번째 질문'),
    ]
})
```

### 한계점

| 한계 | 설명 |
|------|------|
| **수동 관리** | 대화 기록을 직접 관리해야 함 |
| **토큰 제한** | 대화가 길어지면 토큰 한도 초과 |
| **영속성 없음** | 프로그램 종료 시 기록 사라짐 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**LangGraph의 StateGraph**를 사용하여 더 체계적으로 대화 상태를 관리하는 방법을 배웁니다. (02-06번 노트북)